# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [8]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [9]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [3]:
# os.getcwd()

'C:\\Users\\Anete\\PycharmProjects\\dataModellingPostgres'

In [4]:
# os.listdir()

['.idea',
 '.ipynb_checkpoints',
 'create_tables.py',
 'data',
 'etl.ipynb',
 'etl.py',
 'main.py',
 'sql_queries.py',
 'test (2).ipynb',
 'test.ipynb',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'venv',
 '__pycache__']

In [10]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [11]:
song_files = get_files('data/song_data')

In [12]:
filepath = song_files[0]

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  ARD7TVE1187B99BFB1              NaN               NaN   

   artist_location artist_name             song_id             title  \
0  California - LA      Casual  SOMZWCG12A8C13C480  I Didn't Mean To   

    duration  year  
0  218.93179     0

In [7]:
# song_files = pd.DataFrame(song_files)
# pd.set_option('max_colwidth', 800)
# song_files.head()

0
0  C:\Users\Anete\PycharmProjects\dataModellingPostgres\data\song_data\A\A\A\TRAAAAW128F429D538.json
1  C:\Users\Anete\PycharmProjects\dataModellingPostgres\data\song_data\A\A\A\TRAAABD128F429CF47.json
2  C:\Users\Anete\PycharmProjects\dataModellingPostgres\data\song_data\A\A\A\TRAAADZ128F9348C2E.json
3  C:\Users\Anete\PycharmProjects\dataModellingPostgres\data\song_data\A\A\A\TRAAAEF128F4273421.json
4  C:\Users\Anete\PycharmProjects\dataModellingPostgres\data\song_data\A\A\A\TRAAAFD128F92F423A.json

In [71]:
# song_data_file = pd.read_json('C:\\Users\\Anete\\PycharmProjects\\dataModellingPostgres\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json', lines=True)
# song_data_file

num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  ARD7TVE1187B99BFB1              NaN               NaN   

   artist_location artist_name             song_id             title  \
0  California - LA      Casual  SOMZWCG12A8C13C480  I Didn't Mean To   

    duration  year  
0  218.93179     0

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [14]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [18]:
cur.execute(songs_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [16]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
cur.execute(artists_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [19]:
log_files = get_files('data/log_data')

In [20]:
filepath = log_files[0]

In [21]:
df = pd.read_json(filepath, lines=True)
df.head()

artist       auth firstName gender  itemInSession lastName     length  \
0     None  Logged In    Walter      M              0     Frye        NaN   
1     None  Logged In    Kaylee      F              0  Summers        NaN   
2  Des'ree  Logged In    Kaylee      F              1  Summers  246.30812   
3     None  Logged In    Kaylee      F              2  Summers        NaN   
4  Mr Oizo  Logged In    Kaylee      F              3  Summers  144.03873   

  level                           location method      page   registration  \
0  free  San Francisco-Oakland-Hayward, CA    GET      Home  1540919166796   
1  free        Phoenix-Mesa-Scottsdale, AZ    GET      Home  1540344794796   
2  free        Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong  1540344794796   
3  free        Phoenix-Mesa-Scottsdale, AZ    GET   Upgrade  1540344794796   
4  free        Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong  1540344794796   

   sessionId          song  status             ts  \
0         38          None     200  1541105830796   
1        139          None     200  1541106106796   
2        139  You Gotta Be     200  1541106106796   
3        139          None     200  1541106132796   
4        139       Flat 55     200  1541106352796   

                                                                                                                    userAgent  \
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"   
1             "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   
2             "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   
3             "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   
4             "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   

   userId  
0      39  
1       8  
2       8  
3       8  
4       8

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [22]:
df = df.loc[df['page']=='NextSong']
df.head()

artist       auth firstName gender  itemInSession lastName  \
2            Des'ree  Logged In    Kaylee      F              1  Summers   
4            Mr Oizo  Logged In    Kaylee      F              3  Summers   
5         Tamba Trio  Logged In    Kaylee      F              4  Summers   
6     The Mars Volta  Logged In    Kaylee      F              5  Summers   
7  Infected Mushroom  Logged In    Kaylee      F              6  Summers   

      length level                     location method      page  \
2  246.30812  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   
4  144.03873  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   
5  177.18812  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   
6  380.42077  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   
7  440.26730  free  Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong   

    registration  sessionId                          song  status  \
2  1540344794796        139                  You Gotta Be     200   
4  1540344794796        139                       Flat 55     200   
5  1540344794796        139  Quem Quiser Encontrar O Amor     200   
6  1540344794796        139                     Eriatarka     200   
7  1540344794796        139               Becoming Insane     200   

              ts  \
2  1541106106796   
4  1541106352796   
5  1541106496796   
6  1541106673796   
7  1541107053796   

                                                                                                         userAgent  \
2  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   
4  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   
5  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   
6  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   
7  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"   

   userId  
2       8  
4       8  
5       8  
6       8  
7       8

In [23]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [24]:
time_data = [t, t.dt.year, t.dt.month, t.dt.isocalendar().week, t.dt.weekday, t.dt.day, t.dt.hour]
column_labels = ('start_time', 'year', 'month', 'week', 'weekday', 'day', 'hour')

In [23]:
# print(type(column_labels))

<class 'tuple'>


In [28]:
# time_df_test = dict(zip(column_labels, time_data))
# print(time_df_test)

{'start_time': 2    2018-11-01 21:01:46.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns], 'year': 2     2018
4     2018
5     2018
6     2018
7     2018
8     2018
9     2018
10    2018
12    2018
13    2018
14    2018
Name: ts, dtype: int64, 'month': 2     11
4     11
5     11
6     11
7     11
8     11
9     11
10    11
12    11
13    11
14    11
Name: ts, dtype: int64, 'week': 2     44
4     44
5     44
6     44
7     44
8     44
9     44
10    44
12    44
13    44
14    44
Name: week, dtype: UInt32, 'weekday': 2     3
4     3
5     3
6     3
7     3
8     3
9     3
10    3
12    3
13    3
14    3
Name: ts, dtype: int64, 'day': 2     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
12    1
13    1
14  

In [25]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

start_time  year  month  week  weekday  day  hour
2 2018-11-01 21:01:46.796  2018     11    44        3    1    21
4 2018-11-01 21:05:52.796  2018     11    44        3    1    21
5 2018-11-01 21:08:16.796  2018     11    44        3    1    21
6 2018-11-01 21:11:13.796  2018     11    44        3    1    21
7 2018-11-01 21:17:33.796  2018     11    44        3    1    21

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [26]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [27]:
user_df = df[['userId', 'firstName','lastName', 'gender', 'level']].drop_duplicates(keep='first')
user_df_unique = user_df.drop_duplicates(keep='first')

user_df.head()

userId firstName lastName gender level
2        8    Kaylee  Summers      F  free
10      10    Sylvie     Cruz      F  free
12      26      Ryan    Smith      M  free
14     101    Jayden      Fox      M  free

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [28]:
for i, row in user_df_unique.iterrows():
    cur.execute(users_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [29]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(song_plays_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [30]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.